In [1]:
import itertools
import time

import requests
import datetime


class config:
    moodle_host = "http://edu.mmcs.sfedu.ru"
    username = ""
    password = ""
    service = "moodle_mobile_app"


class MoodleApi:
    def __init__(self, host, username, password, service):
        self.host = host
        self.username = username
        self.password = password
        self.service = service
        self.token = None
        self.info = None

    def login(self):
        params = {
            'username': self.username,
            'password': self.password,
            'service': self.service,
        }

        print(f'login to "{self.host}"')
        response = requests.get(f'{self.host}/login/token.php', params=params)
        if response.status_code == 200 and 'error' not in response.json().keys():
            self.token = response.json()['token']
            self.info = self.call('core_webservice_get_site_info')
        else:
            raise Exception(response.json())

    def call(self, function_name, **external_params):
        if self.token is None:
            self.login()

        call_params = {
            'moodlewsrestformat': 'json',
            'moodlewssettingfilter': 'true',
            'moodlewssettingfileurl': 'true',
            'wsfunction': function_name,
            'wstoken': self.token,
            **external_params
        }
#         print(f'call "{function_name}", params = {call_params}')
        response = requests.get(f'{self.host}/webservice/rest/server.php',
                                params=call_params)
        return response.json()


class Calendar:
    calendar_function = 'core_calendar_get_calendar_monthly_view'

    def __init__(self, api: MoodleApi):
        self.api = api
        self.events = []

    def update(self):
        self.events = []

        today = datetime.datetime.fromtimestamp(time.time())
        params = {'year': today.year, 'month': today.month, }
        calendar = self.api.call(self.calendar_function, **params)
        days = list(map(lambda w: w['days'], calendar['weeks']))
        days = list(itertools.chain(*days))
        events = list(filter(lambda e: e != [], map(lambda d: d['events'], days)))
        events = list(itertools.chain.from_iterable(events))

        self.events = events
        return self.events

    def print_all(self):
        print('events in this month:')
        for event in self.events:
            deadline = datetime.datetime.fromtimestamp(event["timestart"])
            print(f'\tcourse "{event["course"]["shortname"]}", '
                  f'name "{event["name"]}", '
                  f'deadline {deadline}')


In [2]:
api = MoodleApi(config.moodle_host, config.username,
                config.password, config.service)

api.login()
print(f'site_description = {api.info["sitename"]}')
print(f'user = {api.info["fullname"]}')

calendar = Calendar(api)
calendar.update()
calendar.print_all()


login to "http://edu.mmcs.sfedu.ru"


Exception: {'error': 'Неверный логин или пароль, попробуйте заново.', 'errorcode': 'invalidlogin', 'stacktrace': None, 'debuginfo': None, 'reproductionlink': None}